In [1]:
# should already be activated, but just a reminder where ya are :)  
# !conda activate wool_sucking_nn

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats 

%matplotlib inline

In [3]:
# import data
excel_file = pd.read_excel('..\Data\cleaned_and_split_data.xlsx', sheet_name=None)

# Unpack the dataframes into separate variables
X_train = excel_file['X_train']
X_test = excel_file['X_test']
y_train = excel_file['y_train']
y_test = excel_file['y_test']

In [4]:
print(X_train.shape)
X_train.head()

(3965, 7)


,Neuter_status,Breed_group,Aggression_owner,Aggression_cats,Shyness_novel,Shyness_strangers,Grooming
0,1,SIB,1,2,4,4,3
1,1,HCS,1,2,4,4,2
2,0,ORI,1,1,1,1,1
3,1,MCO,1,1,2,2,3
4,1,HCS,1,1,2,1,2


In [5]:
print(X_test.shape)
X_test.head()

(1700, 7)


,Neuter_status,Breed_group,Aggression_owner,Aggression_cats,Shyness_novel,Shyness_strangers,Grooming
0,1,SIB,1,3,3,1,1
1,1,MCO,1,1,2,1,1
2,1,RUS,1,1,4,4,1
3,0,EUR,2,2,2,2,1
4,1,NFO,1,2,3,2,1


In [6]:
print(y_train.shape)
y_train.head()

(3965, 1)


,Wool_sucking_binary
0,0
1,0
2,0
3,0
4,0


In [7]:
print(y_test.shape)
y_test.head()

(1700, 1)


,Wool_sucking_binary
0,1
1,1
2,1
3,0
4,1


## To record model metrics

In [8]:
# dataframe for scores amongst models
master_scores = pd.DataFrame(columns=['Model','Recall', 'F1', 'Precision', 'Accuracy'])

In [9]:
# add_to_master(model_description, y_true, y_predicted, binary=True)
def add_to_master(model_description, y_test, y_pred, binary=True):
    '''
    Adds a new row to the running score DataFrame `master_scores` 
    and functions for adding both binary and multiclass scores. 
    '''
    global master_scores
    if binary == True:
        # Evalution Metrics
        accuracy = accuracy_score(y_test, y_pred)
        precision = precision_score(y_test, y_pred)
        recall = recall_score(y_test, y_pred)
        f1 = f1_score(y_test, y_pred)

        # add to master_scores
        new_row = {
            'Model': model_description,
            'Recall': round(recall,3), 
            'F1': round(f1, 3), 
            'Precision': round(precision, 3), 
            'Accuracy': round(accuracy, 3)}
        
        master_scores = master_scores.append(new_row, ignore_index=True)   
    
    else:
        accuracy = accuracy_score(y_test, y_pred)
        precision = precision_score(y_test, y_pred, average='macro') # I chose macro since we have less of the sucklers 
        recall = recall_score(y_test, y_pred, average='macro')       # and I want their accuracy to be meaningful
        f1 = f1_score(y_test, y_pred, average='macro')


        # add to master_scores
        new_row = {
            'Model': model_description,
            'Recall': round(recall,3), 
            'F1': round(f1, 3), 
            'Precision': round(precision, 3), 
            'Accuracy': round(accuracy, 3)}
        master_scores = master_scores.append(new_row, ignore_index=True)
    return master_scores

# Prep for tensorflow.keras

In [10]:
import tensorflow as tf
from tensorflow import keras
from sklearn.metrics import accuracy_score,precision_score, recall_score, f1_score, confusion_matrix
from sklearn.preprocessing import StandardScaler
# for reproducibility 
tf.random.set_seed(42)
np.random.seed(42)

In [11]:
# one-hot breed 
X_train_encoded = pd.get_dummies(X_train, columns=['Breed_group'])
X_test_encoded = pd.get_dummies(X_test, columns=['Breed_group'])

In [12]:
X_train_encoded

,Neuter_status,Aggression_owner,Aggression_cats,Shyness_novel,Shyness_strangers,Grooming,Breed_group_ABY,Breed_group_BEN,Breed_group_BRI,Breed_group_BUR,...,Breed_group_MCO,Breed_group_NFO,Breed_group_ORI,Breed_group_PER,Breed_group_RAG,Breed_group_RUS,Breed_group_SBI,Breed_group_SIB,Breed_group_TUV,Breed_group_other
0,1,1,2,4,4,3,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
1,1,1,2,4,4,2,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,1,1,1,1,1,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
3,1,1,1,2,2,3,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
4,1,1,1,2,1,2,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3960,1,1,3,1,1,2,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
3961,1,2,2,2,1,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3962,1,1,1,3,4,1,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
3963,1,1,1,4,4,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


## Plain Jane

In [13]:
model = keras.Sequential([
    keras.layers.Dense(64, activation='relu', input_shape=(len(X_train_encoded.columns),)),
    keras.layers.Dense(64, activation='relu'),
    keras.layers.Dense(1, activation='sigmoid')
])

In [14]:
def recall(y_true, y_pred):
    true_positives = tf.keras.backend.sum(tf.keras.backend.round(tf.keras.backend.clip(y_true * y_pred, 0, 1)))
    possible_positives = tf.keras.backend.sum(tf.keras.backend.round(tf.keras.backend.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + tf.keras.backend.epsilon())
    return recall

tf.keras.utils.get_custom_objects()['recall'] = recall

In [15]:
model.compile(optimizer='adam', 
             loss='binary_crossentropy',
             metrics=['accuracy', 'recall'])

In [16]:
model.fit(X_train_encoded, y_train, epochs=10, batch_size=32)

Epoch 1/10
124/124 [==============================] - 1s 933us/step - loss: 0.6306 - accuracy: 0.6673 - recall: 0.0807
Epoch 2/10
124/124 [==============================] - 0s 933us/step - loss: 0.6026 - accuracy: 0.6931 - recall: 0.1470
Epoch 3/10
124/124 [==============================] - 0s 934us/step - loss: 0.5938 - accuracy: 0.6941 - recall: 0.1884
Epoch 4/10
124/124 [==============================] - 0s 973us/step - loss: 0.5900 - accuracy: 0.6921 - recall: 0.1557
Epoch 5/10
124/124 [==============================] - 0s 916us/step - loss: 0.5856 - accuracy: 0.6951 - recall: 0.1727
Epoch 6/10
124/124 [==============================] - 0s 933us/step - loss: 0.5837 - accuracy: 0.6974 - recall: 0.2135
Epoch 7/10
124/124 [==============================] - 0s 916us/step - loss: 0.5829 - accuracy: 0.6946 - recall: 0.2147
Epoch 8/10
124/124 [==============================] - 0s 924us/step - loss: 0.5777 - accuracy: 0.6984 - recall: 0.2063
Epoch 9/10
124/124 [============================

In [17]:
test_loss, test_acc, test_recall = model.evaluate(X_test_encoded, y_test)
print('Test Accuracy:', test_acc)
print('Test Loss:', test_loss)
print('Test Recall:', test_recall)

54/54 [==============================] - 0s 734us/step - loss: 0.5857 - accuracy: 0.6812 - recall: 0.2399
Test Accuracy: 0.681176483631134
Test Loss: 0.585716962814331
Test Recall: 0.23990733921527863


In [18]:
y_pred = model.predict(X_test_encoded)
y_pred = np.round(y_pred).astype(int)

cm = confusion_matrix(y_test, y_pred)
# add_to_master(model_description, y_true, y_predicted, binary=True)
add_to_master("1 hidden, 64 nodes per, adam", y_test, y_pred)
cm

54/54 [==============================] - 0s 621us/step


C:\Users\gwent\AppData\Local\Temp\ipykernel_2572\1005917253.py:23: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  master_scores = master_scores.append(new_row, ignore_index=True)


array([[1021,  138],
       [ 404,  137]], dtype=int64)

In [19]:
master_scores

,Model,Recall,F1,Precision,Accuracy
0,"1 hidden, 64 nodes per, adam",0.253,0.336,0.498,0.681


Not doing too hot, but also seems to be doing better than initial models in Cat-pstone 1. 

## Adagrad optimizer

In [20]:
model2 = keras.Sequential([
    keras.layers.Dense(64, activation='relu', input_shape=(len(X_train_encoded.columns),)),
    keras.layers.Dense(64, activation='relu'),
    keras.layers.Dense(1, activation='sigmoid')
])

In [21]:
# Adagrad better for when feataures vary in magnitude and frequency 
optimizer = tf.keras.optimizers.Adagrad(learning_rate=0.01)
model2.compile(optimizer=optimizer, 
             loss='binary_crossentropy',
             metrics=['accuracy', 'recall'])

In [22]:
model2.fit(X_train_encoded, y_train, epochs=10, batch_size=32)

Epoch 1/10
124/124 [==============================] - 0s 835us/step - loss: 0.6422 - accuracy: 0.6817 - recall: 0.0000e+00
Epoch 2/10
124/124 [==============================] - 0s 860us/step - loss: 0.6301 - accuracy: 0.6822 - recall: 0.0025
Epoch 3/10
124/124 [==============================] - 0s 851us/step - loss: 0.6219 - accuracy: 0.6812 - recall: 0.0219
Epoch 4/10
124/124 [==============================] - 0s 851us/step - loss: 0.6164 - accuracy: 0.6845 - recall: 0.0324
Epoch 5/10
124/124 [==============================] - 0s 843us/step - loss: 0.6118 - accuracy: 0.6865 - recall: 0.0466
Epoch 6/10
124/124 [==============================] - 0s 868us/step - loss: 0.6081 - accuracy: 0.6835 - recall: 0.0629
Epoch 7/10
124/124 [==============================] - 0s 868us/step - loss: 0.6052 - accuracy: 0.6898 - recall: 0.0906
Epoch 8/10
124/124 [==============================] - 0s 868us/step - loss: 0.6027 - accuracy: 0.6908 - recall: 0.0880
Epoch 9/10
124/124 [========================

In [23]:
test_loss, test_acc, test_recall = model2.evaluate(X_test_encoded, y_test)
print('Test Accuracy:', test_acc)
print('Test Loss:', test_loss)
print('Test Recall:', test_recall)

54/54 [==============================] - 0s 734us/step - loss: 0.5993 - accuracy: 0.6818 - recall: 0.1463
Test Accuracy: 0.6817647218704224
Test Loss: 0.5992920994758606
Test Recall: 0.14630085229873657


In [24]:
y_pred = model2.predict(X_test_encoded)
y_pred = np.round(y_pred).astype(int)

cm = confusion_matrix(y_test, y_pred)
add_to_master("1 hidden, 64 nodes per, adagrad", y_test, y_pred)
cm

54/54 [==============================] - 0s 602us/step


C:\Users\gwent\AppData\Local\Temp\ipykernel_2572\1005917253.py:23: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  master_scores = master_scores.append(new_row, ignore_index=True)


array([[1076,   83],
       [ 458,   83]], dtype=int64)

## Scaled with Plain Jane and Adagrad

In [25]:
scaler = StandardScaler()
scaler.fit(X_train_encoded)

StandardScaler()

In [26]:
X_train_scaled_encoded = scaler.transform(X_train_encoded)
X_test_scaled_encoded = scaler.transform(X_test_encoded)

In [27]:
X_train_scaled_encoded.shape

(3965, 25)

In [28]:
# Plain Jane + scaled
model3 = keras.Sequential([
    keras.layers.Dense(64, activation='relu', input_shape=(25,)),
    keras.layers.Dense(64, activation='relu'),
    keras.layers.Dense(1, activation='sigmoid')
])

model3.compile(optimizer='adam', 
             loss='binary_crossentropy',
             metrics=['accuracy', 'recall'])

model3.fit(X_train_scaled_encoded, y_train, epochs=10, batch_size=32)

test_loss, test_acc, test_recall = model3.evaluate(X_test_scaled_encoded, y_test)
print('Test Accuracy:', test_acc)
print('Test Loss:', test_loss)
print('Test Recall:', test_recall)

y_pred = model3.predict(X_test_scaled_encoded)
y_pred = np.round(y_pred).astype(int)

cm = confusion_matrix(y_test, y_pred)
add_to_master("1 hidden, 64 nodes per, adam, scaled", y_test, y_pred)
cm

Epoch 1/10
124/124 [==============================] - 0s 892us/step - loss: 0.6161 - accuracy: 0.6794 - recall: 0.0632
Epoch 2/10
124/124 [==============================] - 0s 909us/step - loss: 0.5912 - accuracy: 0.6918 - recall: 0.1441
Epoch 3/10
124/124 [==============================] - 0s 916us/step - loss: 0.5835 - accuracy: 0.6999 - recall: 0.2176
Epoch 4/10
124/124 [==============================] - 0s 908us/step - loss: 0.5796 - accuracy: 0.7006 - recall: 0.1890
Epoch 5/10
124/124 [==============================] - 0s 924us/step - loss: 0.5744 - accuracy: 0.6999 - recall: 0.2100
Epoch 6/10
124/124 [==============================] - 0s 933us/step - loss: 0.5737 - accuracy: 0.7026 - recall: 0.2370
Epoch 7/10
124/124 [==============================] - 0s 916us/step - loss: 0.5719 - accuracy: 0.7090 - recall: 0.2531
Epoch 8/10
124/124 [==============================] - 0s 916us/step - loss: 0.5682 - accuracy: 0.7077 - recall: 0.2437
Epoch 9/10
124/124 [============================

C:\Users\gwent\AppData\Local\Temp\ipykernel_2572\1005917253.py:23: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  master_scores = master_scores.append(new_row, ignore_index=True)


array([[1003,  156],
       [ 386,  155]], dtype=int64)

In [29]:
# adagrad + scaled 
model4 = keras.Sequential([
    keras.layers.Dense(64, activation='relu', input_shape=(25,)),
    keras.layers.Dense(64, activation='relu'),
    keras.layers.Dense(1, activation='sigmoid')
])

# Adagrad better for when feataures vary in magnitude and frequency 
optimizer = tf.keras.optimizers.Adagrad(learning_rate=0.01)
model4.compile(optimizer=optimizer, 
             loss='binary_crossentropy',
             metrics=['accuracy', 'recall'])

model4.fit(X_train_scaled_encoded, y_train, epochs=10, batch_size=32)

test_loss, test_acc, test_recall = model4.evaluate(X_test_scaled_encoded, y_test)
print('Test Accuracy:', test_acc)
print('Test Loss:', test_loss)
print('Test Recall:', test_recall)

y_pred = model4.predict(X_test_scaled_encoded)
y_pred = np.round(y_pred).astype(int)

cm = confusion_matrix(y_test, y_pred)
add_to_master("1 hidden, 64 nodes per, adagrad, scaled", y_test, y_pred)
cm

Epoch 1/10
124/124 [==============================] - 0s 860us/step - loss: 0.6376 - accuracy: 0.6515 - recall: 0.0880
Epoch 2/10
124/124 [==============================] - 0s 876us/step - loss: 0.6039 - accuracy: 0.6847 - recall: 0.0294
Epoch 3/10
124/124 [==============================] - 0s 868us/step - loss: 0.5958 - accuracy: 0.6860 - recall: 0.1069
Epoch 4/10
124/124 [==============================] - 0s 851us/step - loss: 0.5913 - accuracy: 0.6895 - recall: 0.1154
Epoch 5/10
124/124 [==============================] - 0s 908us/step - loss: 0.5882 - accuracy: 0.6926 - recall: 0.1382
Epoch 6/10
124/124 [==============================] - 0s 884us/step - loss: 0.5857 - accuracy: 0.6933 - recall: 0.1528
Epoch 7/10
124/124 [==============================] - 0s 876us/step - loss: 0.5837 - accuracy: 0.6941 - recall: 0.1596
Epoch 8/10
124/124 [==============================] - 0s 924us/step - loss: 0.5823 - accuracy: 0.6953 - recall: 0.1794
Epoch 9/10
124/124 [============================

C:\Users\gwent\AppData\Local\Temp\ipykernel_2572\1005917253.py:23: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  master_scores = master_scores.append(new_row, ignore_index=True)


array([[1063,   96],
       [ 450,   91]], dtype=int64)

In [30]:
master_scores.sort_values('Recall', ascending=False)

,Model,Recall,F1,Precision,Accuracy
2,"1 hidden, 64 nodes per, adam, scaled",0.287,0.364,0.498,0.681
0,"1 hidden, 64 nodes per, adam",0.253,0.336,0.498,0.681
3,"1 hidden, 64 nodes per, adagrad, scaled",0.168,0.250,0.487,0.679
1,"1 hidden, 64 nodes per, adagrad",0.153,0.235,0.500,0.682


In [31]:
def create_mlp(nodes, optimizer):
    # Define the model architecture
    model = keras.Sequential([
    keras.layers.Dense(nodes, activation='relu', input_shape=(25,)),
    keras.layers.Dense(nodes, activation='relu'),
    keras.layers.Dense(1, activation='sigmoid')
])

    model.compile(optimizer=optimizer, 
             loss='binary_crossentropy',
             metrics=['accuracy', 'recall'])
    
    return model

In [32]:
model5 = create_mlp(100, 'adam')

In [33]:
model5.fit(X_train_scaled_encoded, y_train, epochs=10, batch_size=32)

test_loss, test_acc, test_recall = model5.evaluate(X_test_scaled_encoded, y_test)
print('Test Accuracy:', test_acc)
print('Test Loss:', test_loss)
print('Test Recall:', test_recall)

y_pred = model5.predict(X_test_scaled_encoded)
y_pred = np.round(y_pred).astype(int)

cm = confusion_matrix(y_test, y_pred)
add_to_master("1 hidden, 100 nodes, adam", y_test, y_pred)
cm

Epoch 1/10
124/124 [==============================] - 0s 1ms/step - loss: 0.6071 - accuracy: 0.6820 - recall: 0.1187
Epoch 2/10
124/124 [==============================] - 0s 1ms/step - loss: 0.5863 - accuracy: 0.6936 - recall: 0.1819
Epoch 3/10
124/124 [==============================] - 0s 1ms/step - loss: 0.5803 - accuracy: 0.6948 - recall: 0.2307
Epoch 4/10
124/124 [==============================] - 0s 1ms/step - loss: 0.5769 - accuracy: 0.7024 - recall: 0.2178
Epoch 5/10
124/124 [==============================] - 0s 1ms/step - loss: 0.5709 - accuracy: 0.7042 - recall: 0.2272
Epoch 6/10
124/124 [==============================] - 0s 1ms/step - loss: 0.5709 - accuracy: 0.7090 - recall: 0.2599
Epoch 7/10
124/124 [==============================] - 0s 1ms/step - loss: 0.5685 - accuracy: 0.7047 - recall: 0.2732
Epoch 8/10
124/124 [==============================] - 0s 1ms/step - loss: 0.5638 - accuracy: 0.7148 - recall: 0.2734
Epoch 9/10
124/124 [==============================] - 0s 1ms/ste

C:\Users\gwent\AppData\Local\Temp\ipykernel_2572\1005917253.py:23: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  master_scores = master_scores.append(new_row, ignore_index=True)


array([[984, 175],
       [399, 142]], dtype=int64)

In [34]:
model6 = create_mlp(32, 'adam')
model6.fit(X_train_scaled_encoded, y_train, epochs=10, batch_size=32)

test_loss, test_acc, test_recall = model6.evaluate(X_test_scaled_encoded, y_test)
print('Test Accuracy:', test_acc)
print('Test Loss:', test_loss)
print('Test Recall:', test_recall)

y_pred = model6.predict(X_test_scaled_encoded)
y_pred = np.round(y_pred).astype(int)

cm = confusion_matrix(y_test, y_pred)
add_to_master("1 hidden, 32 nodes, adam", y_test, y_pred)
cm

Epoch 1/10
124/124 [==============================] - 0s 884us/step - loss: 0.6269 - accuracy: 0.6575 - recall: 0.0923
Epoch 2/10
124/124 [==============================] - 0s 892us/step - loss: 0.5956 - accuracy: 0.6916 - recall: 0.0937
Epoch 3/10
124/124 [==============================] - 0s 884us/step - loss: 0.5893 - accuracy: 0.6953 - recall: 0.1816
Epoch 4/10
124/124 [==============================] - 0s 916us/step - loss: 0.5852 - accuracy: 0.6946 - recall: 0.1524
Epoch 5/10
124/124 [==============================] - 0s 908us/step - loss: 0.5814 - accuracy: 0.6986 - recall: 0.1862
Epoch 6/10
124/124 [==============================] - 0s 892us/step - loss: 0.5802 - accuracy: 0.6953 - recall: 0.1899
Epoch 7/10
124/124 [==============================] - 0s 892us/step - loss: 0.5783 - accuracy: 0.6981 - recall: 0.2013
Epoch 8/10
124/124 [==============================] - 0s 892us/step - loss: 0.5761 - accuracy: 0.6996 - recall: 0.2150
Epoch 9/10
124/124 [============================

C:\Users\gwent\AppData\Local\Temp\ipykernel_2572\1005917253.py:23: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  master_scores = master_scores.append(new_row, ignore_index=True)


array([[1049,  110],
       [ 430,  111]], dtype=int64)

In [35]:
model7 = create_mlp(250, 'adam')
model7.fit(X_train_scaled_encoded, y_train, epochs=10, batch_size=32)

test_loss, test_acc, test_recall = model7.evaluate(X_test_scaled_encoded, y_test)
print('Test Accuracy:', test_acc)
print('Test Loss:', test_loss)
print('Test Recall:', test_recall)

y_pred = model7.predict(X_test_scaled_encoded)
y_pred = np.round(y_pred).astype(int)

cm = confusion_matrix(y_test, y_pred)
add_to_master("1 hidden, 250 nodes, adam", y_test, y_pred)
cm

Epoch 1/10
124/124 [==============================] - 1s 1ms/step - loss: 0.6080 - accuracy: 0.6810 - recall: 0.1522
Epoch 2/10
124/124 [==============================] - 0s 1ms/step - loss: 0.5880 - accuracy: 0.6971 - recall: 0.2107
Epoch 3/10
124/124 [==============================] - 0s 1ms/step - loss: 0.5801 - accuracy: 0.6974 - recall: 0.2232
Epoch 4/10
124/124 [==============================] - 0s 1ms/step - loss: 0.5775 - accuracy: 0.7014 - recall: 0.2026
Epoch 5/10
124/124 [==============================] - 0s 1ms/step - loss: 0.5690 - accuracy: 0.7087 - recall: 0.2542
Epoch 6/10
124/124 [==============================] - 0s 1ms/step - loss: 0.5703 - accuracy: 0.7122 - recall: 0.2667
Epoch 7/10
124/124 [==============================] - 0s 1ms/step - loss: 0.5680 - accuracy: 0.7107 - recall: 0.2744
Epoch 8/10
124/124 [==============================] - 0s 1ms/step - loss: 0.5621 - accuracy: 0.7110 - recall: 0.2718
Epoch 9/10
124/124 [==============================] - 0s 1ms/ste

C:\Users\gwent\AppData\Local\Temp\ipykernel_2572\1005917253.py:23: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  master_scores = master_scores.append(new_row, ignore_index=True)


array([[990, 169],
       [396, 145]], dtype=int64)

In [36]:
# definitely overfits
model8 = create_mlp(300, 'adam')
model8.fit(X_train_scaled_encoded, y_train, epochs=10, batch_size=32)

test_loss, test_acc, test_recall = model8.evaluate(X_test_scaled_encoded, y_test)
print('Test Accuracy:', test_acc)
print('Test Loss:', test_loss)
print('Test Recall:', test_recall)

y_pred = model8.predict(X_test_scaled_encoded)
y_pred = np.round(y_pred).astype(int)

cm = confusion_matrix(y_test, y_pred)
add_to_master("1 hidden, 300 nodes, adam", y_test, y_pred)
cm

Epoch 1/10
124/124 [==============================] - 1s 1ms/step - loss: 0.6067 - accuracy: 0.6802 - recall: 0.1642
Epoch 2/10
124/124 [==============================] - 0s 1ms/step - loss: 0.5882 - accuracy: 0.6951 - recall: 0.2114
Epoch 3/10
124/124 [==============================] - 0s 1ms/step - loss: 0.5803 - accuracy: 0.7001 - recall: 0.2361
Epoch 4/10
124/124 [==============================] - 0s 1ms/step - loss: 0.5782 - accuracy: 0.7001 - recall: 0.2039
Epoch 5/10
124/124 [==============================] - 0s 1ms/step - loss: 0.5707 - accuracy: 0.7077 - recall: 0.2467
Epoch 6/10
124/124 [==============================] - 0s 1ms/step - loss: 0.5718 - accuracy: 0.7090 - recall: 0.2574
Epoch 7/10
124/124 [==============================] - 0s 1ms/step - loss: 0.5695 - accuracy: 0.7074 - recall: 0.2654
Epoch 8/10
124/124 [==============================] - 0s 1ms/step - loss: 0.5632 - accuracy: 0.7100 - recall: 0.2619
Epoch 9/10
124/124 [==============================] - 0s 1ms/ste

C:\Users\gwent\AppData\Local\Temp\ipykernel_2572\1005917253.py:23: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  master_scores = master_scores.append(new_row, ignore_index=True)


array([[996, 163],
       [395, 146]], dtype=int64)

In [37]:
master_scores.sort_values('Recall', ascending=False)

,Model,Recall,F1,Precision,Accuracy
2,"1 hidden, 64 nodes per, adam, scaled",0.287,0.364,0.498,0.681
7,"1 hidden, 300 nodes, adam",0.270,0.344,0.472,0.672
6,"1 hidden, 250 nodes, adam",0.268,0.339,0.462,0.668
4,"1 hidden, 100 nodes, adam",0.262,0.331,0.448,0.662
0,"1 hidden, 64 nodes per, adam",0.253,0.336,0.498,0.681
5,"1 hidden, 32 nodes, adam",0.205,0.291,0.502,0.682
3,"1 hidden, 64 nodes per, adagrad, scaled",0.168,0.250,0.487,0.679
1,"1 hidden, 64 nodes per, adagrad",0.153,0.235,0.500,0.682


Okay so we are pretty similar in performance to the logistic regression models from cat-pstone 1 on every metric except recall (and the part it plays in F1, of course). Let's see if over or undersampling with the top performing model architecture (model1). 